# Getting started with prompty


**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using prompty and visualize the trace of your application.
- batch run prompty against multi lines of data.


## 1. Execute a Prompty

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [ ]:
with open("basic.prompty") as fin:
    print(fin.read())

Note: before running below cell, please configure required environment variable `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT` by create an `.env` file. Please refer to `../.env.example` as an template.


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
#create a .env file with the following variables and replace with your values
AISTUDIO_AZURE_OPENAI_KEY = os.getenv("AISTUDIO_AZURE_OPENAI_KEY")
AISTUDIO_AZURE_OPENAI_ENDPOINT = os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME")
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
AZURE_AISTUDIO_PROJECT_RESOURCE_GROUP = os.getenv("AZURE_AISTUDIO_PROJECT_RESOURCE_GROUP")
AZURE_AISTUDIO_PROJECT_NAME = os.getenv("AZURE_AISTUDIO_PROJECT_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

Note: before running below cell, please configure required environment variable AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT by create an .env file. Please refer to ../.env.example as an template.

In [3]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load(source="basic.prompty")

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

You can override configuration with `AzureOpenAIModelConfiguration` and `OpenAIModelConfiguration`.

In [4]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration

# override configuration with AzureOpenAIModelConfiguration - in this case I am using the same model just to show how to use it
configuration = AzureOpenAIModelConfiguration(
    azure_endpoint=AISTUDIO_AZURE_OPENAI_ENDPOINT,  # Use ${env:<ENV_NAME>} to surround the environment variable name.
    api_key=AISTUDIO_AZURE_OPENAI_KEY,
    azure_deployment=AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME,
)


override_model = {"configuration": configuration, "parameters": {"max_tokens": 512}}

# load prompty as a flow
f = Prompty.load(source="basic.prompty", model=override_model)

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

### Visualize trace by using start_trace

In [5]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Starting prompt flow service...
You can stop the prompt flow service with the following command:'pf service stop'.



Re-run below cell will collect a trace in trace UI.

In [6]:
# rerun the function, which will be recorded in the trace
question = "What is the capital of Japan?"
ground_truth = "Tokyo"
result = f(question=question)
result

You can view the trace detail from the following URL:
http://127.0.0.1:23334/v1.0/ui/traces/?#collection=5-E2E&uiTraceId=0x6b6b7299da75183314d713f248b4cf70
You can view the trace detail from the following URL:
http://127.0.0.1:23334/v1.0/ui/traces/?#collection=5-E2E&uiTraceId=0x4f3ce3826faa0619fbd9042418866777


'The capital of Japan is Tokyo.'

### Eval the result 

Note: the eval flow returns a `json_object`.

In [7]:
# load prompty as a flow
eval_flow = Prompty.load("eval.prompty")
# execute the flow as function
result = eval_flow(question=question, ground_truth=ground_truth, answer=result)
result

{'score': 5,
 'explanation': 'The answer correctly identifies Tokyo as the capital of Japan.'}

## 2. Batch run with multi-line data


In [8]:
from promptflow.client import PFClient

pf = PFClient()

In [9]:
flow = "./basic.prompty"  # path to the prompty file
data = "./data.jsonl"  # path to the data file

# create run with the flow and data
base_run = pf.run(
    flow=flow,
    data=data,
    column_mapping={
        "question": "${data.question}",
    },
    stream=True,
)

[2024-09-24 18:16:14 +0300][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run n5_e2e_20240924_181612_709700, log path: C:\Users\dschlesinger\.promptflow\.runs\n5_e2e_20240924_181612_709700\logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23334/v1.0/ui/traces/?#run=n5_e2e_20240924_181612_709700
2024-09-24 18:16:14 +0300   10088 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-09-24 18:16:25 +0300   10088 execution.bulk     INFO     Current system's available memory is 25331.015625MB, memory consumption of current process is 186.58984375MB, estimated available worker count is 25331.015625/186.58984375 = 135
2024-09-24 18:16:25 +0300   10088 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 135}.
2024-09-24 18:17:07 +0300   10088 execution.bulk     INFO     Process name(SpawnProcess-2)-Process id(24012)-Line number(0) start execution.
2024-09-24 18:17:07 +0300   10088 execution.bulk     INFO     Process name(SpawnProcess-3)-Pro

In [10]:
details = pf.get_details(base_run)
details.head(10)

,inputs.question,inputs.line_number,outputs.output
0,What is capital of France?,0,The capital of France is Paris.
1,What is the meaning of life?,1,The meaning of life is a philosophical questio...
2,What are the planets in Sun system?,2,"The planets in the Solar System are Mercury, V..."


## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually using LLM assert the produced output matches certain expectation. 

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [11]:
eval_prompty = "./eval.prompty"

eval_run = pf.run(
    flow=eval_prompty,
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "question": "${data.question}",
        "answer": "${run.outputs.output}",  
        "ground_truth": "${data.ground_truth}",
    },
    stream=True,
)

[2024-09-24 18:18:09 +0300][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run n5_e2e_20240924_181809_802652, log path: C:\Users\dschlesinger\.promptflow\.runs\n5_e2e_20240924_181809_802652\logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23334/v1.0/ui/traces/?#run=n5_e2e_20240924_181809_802652
2024-09-24 18:18:09 +0300   10088 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-09-24 18:18:09 +0300   10088 execution.bulk     INFO     Current system's available memory is 25342.53125MB, memory consumption of current process is 191.46484375MB, estimated available worker count is 25342.53125/191.46484375 = 132
2024-09-24 18:18:09 +0300   10088 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 132}.
2024-09-24 18:18:17 +0300   10088 execution.bulk     INFO     Process name(SpawnProcess-8)-Process id(34316)-Line number(0) start execution.
2024-09-24 18:18:17 +0300   10088 execution.bulk     INFO     Process name(SpawnProcess-7)-Proce

In [24]:
details = pf.get_details(eval_run)
details.head(10)

,inputs.question,inputs.answer,inputs.ground_truth,inputs.line_number,outputs.score,outputs.explanation
0,What is capital of France?,The capital of France is Paris.,Paris,0,5,The answer correctly identifies Paris as the c...
1,What is the meaning of life?,The meaning of life is a philosophical questio...,The meaning of life is subjective and can vary...,1,5,The answer comprehensively addresses the philo...
2,What are the planets in Sun system?,"The planets in the Solar System are Mercury, V...","The planets in the Solar System are Mercury, V...",2,5,The answer correctly lists all the planets in ...


In [ ]:
# visualize run using ui
pf.visualize([base_run, eval_run])

## Next steps

By now you've successfully run your first prompt flow and even did evaluation on it. That's great!

You can check out more examples:
- [Basic Chat](https://github.com/microsoft/promptflow/tree/main/examples/prompty/chat-basic): demonstrates how to create a chatbot that can remember previous interactions and use the conversation history to generate next message.